In [1]:

import sys; sys.path.insert(0, '../')
import pickle as pkl
import numpy as np
import pandas as pd
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

# Create Forward Model

In [2]:
fwd = forward.create_forward_model()

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.2s remaining:    2.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.2s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


In [3]:

import plotly.figure_factory as FF
import numpy as np
import mne

def brain_plotly(y, fwd):
    ''' takes triangulated mesh, list of coordinates and a vector of brain activity and plots a plotly triangulated surface '''

    _, _, pos, tris = unpack_fwd(fwd)
    # Concatenate tris so that it covers the whole brain
    tmp = tris + int(pos.shape[0]/2)
    new_tris = np.concatenate([tris, tmp], axis=0)
    # Calculate the true value for each triangle (which is the mean of the triangle's vertices)
    colors = []
    for tri in new_tris:
        positions = pos[tri, :]
        indices = []
        for j in positions:
            indices.append(np.where((pos == j).all(axis=1))[0][0])
        colors.append(np.mean(y[indices]))


    x = pos[:, 0]
    y = pos[:, 1]
    z = pos[:, 2]
    ## Plot
    fig1 = FF.create_trisurf(x=x, y=y, z=z,
                            simplices=new_tris,
                            title="Simulated brain activity",
                            color_func=colors,
                            aspectratio=dict(x=1, y=1, z=1),
                            )
    fig1['layout']['height'] = 650
    fig1['layout']['width'] = 720
    return fig1, colors


def unpack_fwd(fwd):
    """ Helper function that extract the most important data structures from the 
    mne.Forward object
    Parameters
    ----------
    fwd : mne.Forward
        The forward model object
    Return
    ------
    fwd_fixed : mne.Forward
        Forward model for fixed dipole orientations
    leadfield : numpy.ndarray
        The leadfield (gain matrix)
    pos : numpy.ndarray
        The positions of dipoles in the source model
    tris : numpy.ndarray
        The triangles that describe the source mmodel
    neighbors : numpy.ndarray
        the neighbors of each dipole in the source model
    """
    if fwd['surf_ori']:
        fwd_fixed = fwd
    else:
        fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                                    use_cps=True, verbose=0)
    tris = fwd['src'][0]['use_tris']
    leadfield = fwd_fixed['sol']['data']

    source = fwd['src']
    try:
        subject_his_id = source[0]['subject_his_id']
        pos_left = mne.vertex_to_mni(source[0]['vertno'], 0, subject_his_id, verbose=0)
        pos_right = mne.vertex_to_mni(source[1]['vertno'],  1, subject_his_id, verbose=0)
    except:
        subject_his_id = 'fsaverage'
        pos_left = mne.vertex_to_mni(source[0]['vertno'], 0, subject_his_id, verbose=0)
        pos_right = mne.vertex_to_mni(source[1]['vertno'],  1, subject_his_id, verbose=0)

    pos = np.concatenate([pos_left, pos_right], axis=0)

    return fwd_fixed, leadfield, pos, tris

# Simulate Source

In [4]:
fwd_fixed, leadfield, pos, tris = unpack_fwd(fwd)
pos.shape

(1284, 3)

In [8]:
fwd_fixed, leadfield, pos, tris = unpack_fwd(fwd)
# Pick random position
idx_pos = np.random.choice(np.arange(pos.shape[0]))
center_of_activity = pos[idx_pos]
# Select neighboring voxels
radius = 20  # mm
distance_to_center = np.sqrt(((pos - center_of_activity)**2).sum(axis=1))
# Create source vector
source = np.zeros(pos.shape[0])
source[np.argwhere( distance_to_center <= radius )[:, 0]] = 1

# Plot the Source

In [9]:
# source = np.random.randn(pos.shape[0])

brain = brain_plotly(source, fwd)[0]
brain